## This notebook merges the standalone data from IMDB and TMDB and cleans some of the columns within that dataframe.  

In [1]:
import urllib.request
import gzip
import requests
import json
from loguru import logger
from IPython.display import clear_output
import pandas as pd
import shutil
import numpy as np
import re
from scipy import stats

tmdb_key = "ad63716b3506edd1aaa3aef6c8ebd46b"

Reading in the files we would like to merge.

In [2]:
standalones_imdb_df = pd.read_csv("NewOriginal.csv")
standalones_imdb_df

,imdb_id,Movie Budget,Movie Box Office US/CA,Movie Box Office Worldwide,Movie Length,Movie Rating,Movie Director Names,Movie Director Links,Movie Cast Names,Movie Cast Links,Movie Awards
0,tt0066921,"$2,200,000 (estimated)","$26,617,553","$26,960,374",2h 16m,8.3,Stanley Kubrick,['https://www.imdb.com/name/nm0000040/'],"Malcolm McDowell, Patrick Magee, Michael Bates","['https://www.imdb.com/name/nm0000532/', 'http...",NaN
1,tt0347048,NaN,"$435,395","$11,065,801",2h 1m,7.9,Fatih Akin,['https://www.imdb.com/name/nm0015359/'],"Birol Ünel, Sibel Kekilli, Güven Kiraç","['https://www.imdb.com/name/nm0960379/', 'http...",Shooting Star\nSibel Kekilli
2,tt0138704,"$60,000 (estimated)","$3,221,152","$3,221,152",1h 24m,7.3,Darren Aronofsky,['https://www.imdb.com/name/nm0004716/'],"Sean Gullette, Mark Margolis, Ben Shenkman","['https://www.imdb.com/name/nm0347797/', 'http...",NaN
3,tt0243255,NaN,"$725,854","$765,705",1h 35m,6.4,Catherine Breillat,['https://www.imdb.com/name/nm0106924/'],"Anaïs Reboux, Roxane Mesquida, Libero De Rienzo","['https://www.imdb.com/name/nm0714378/', 'http...",NaN
4,tt0061395,NaN,"$4,063,348","$4,162,021",1h 40m,7.7,Luis Buñuel,['https://www.imdb.com/name/nm0000320/'],"Catherine Deneuve, Jean Sorel, Michel Piccoli","['https://www.imdb.com/name/nm0000366/', 'http...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
6254,tt1361348,NaN,NaN,NaN,11m,NaN,Christian Juers,['https://www.imdb.com/name/nm3280047/'],"Hanna Zehm, Daniella Schoenecker, Anna Carlzon","['https://www.imdb.com/name/nm4323836/', 'http...",NaN
6255,tt24248480,NaN,NaN,NaN,1h 18m,NaN,Brian Paulin,['https://www.imdb.com/name/nm1079061/'],"Mike Mello, Rich George, Tanya Hennesy","['https://www.imdb.com/name/nm14332919/', 'htt...",NaN
6256,tt1015187,NaN,NaN,NaN,2006,7.8,Paul Lamont,['https://www.imdb.com/name/nm2616798/'],"Armand Assante, David Ogden Stiers","['https://www.imdb.com/name/nm0000800/', 'http...",Telecast - Professional Non-Fiction Division: ...
6257,tt5135104,NaN,NaN,NaN,2015,NaN,Erick Gálvez,['https://www.imdb.com/name/nm5658309/'],"Dadiana Cabrera, Giuseppe Calvinisti, Juan Pab...","['https://www.imdb.com/name/nm7671967/', 'http...",NaN


In [3]:
standalones_tmdb_df = pd.read_csv("standalones_tmdb_data.csv", index_col=[0])
standalones_tmdb_df

,Unnamed: 0,title,imdb_id,id,release_date,runtime,vote_average,vote_count,popularity,budget,revenue,genres,original_language,production_companies,production_countries
0,0,A Clockwork Orange,tt0066921,185,1971-12-19,137,8.218,11594,29.221,2200000,26589000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",en,"[{'id': 174, 'logo_path': '/IuAlhI9eVC9Z8UQWOI...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'..."
1,1,Head-On,tt0347048,363,2004-03-11,117,7.453,470,8.132,0,11030861,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",de,"[{'id': 200, 'logo_path': None, 'name': 'Coraz...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is..."
2,2,Pi,tt0138704,473,1998-07-10,84,7.122,1942,10.689,60000,3221152,"[{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...",en,"[{'id': 22566, 'logo_path': None, 'name': 'Har...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,3,Fat Girl,tt0243255,570,2001-03-07,86,6.215,177,11.034,0,0,"[{'id': 18, 'name': 'Drama'}]",fr,"[{'id': 15130, 'logo_path': None, 'name': 'Ura...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso..."
4,4,Belle de Jour,tt0061395,649,1967-05-24,101,7.420,738,17.263,0,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",fr,"[{'id': 386, 'logo_path': None, 'name': 'Paris...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6733,6733,Slaves of Blood,tt1361348,1041835,2008-12-05,39,0.000,0,0.600,0,0,"[{'id': 27, 'name': 'Horror'}]",de,"[{'id': 184555, 'logo_path': None, 'name': 'No...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]"
6734,6734,Kill Me Slowly,tt24248480,1042028,1996-01-01,70,0.000,0,0.600,0,0,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",en,"[{'id': 12680, 'logo_path': None, 'name': 'Mor...","[{'iso_3166_1': 'US', 'name': 'United States o..."
6735,6735,Frank Lloyd Wright's Buffalo,tt1015187,1042569,2006-09-03,56,0.000,0,0.600,0,0,"[{'id': 99, 'name': 'Documentary'}]",en,[],[]
6736,6736,El Señor de Esquipulas,tt5135104,1042763,2015-10-12,80,0.000,0,0.600,0,0,[],en,[],[]


Merging the files using their common IMDB ids.

In [4]:
standalones_df = pd.merge(standalones_tmdb_df, standalones_imdb_df, on="imdb_id")

In [5]:
# this function parses the award data from IMDB to remove unnecessary information 

def clean_awards(df, award_col_name):
    
    def parse_award_list(award):
        # only keeping first line    
        award = award.split("\n")[0].strip()
        # removing things surrounded by brackets
        award = re.sub("\(.*?\)|\[.*?\]","",award)
        return award   

    all_awards = []

    for index, row in df.iterrows():
        remade_awards = row[award_col_name]

        # handling Nans
        if type(remade_awards) == float:
            all_awards.append([])
            continue

        # separating the string by comma 
        awards = remade_awards.split(",")
        
        # removing None values within the comma seperated string
        awards = list(map(parse_award_list,awards))
        awards = list(filter(None, awards))

        # adding awards to list of all awards
        all_awards.append(awards)
  
    keywords = ["Best", "Most", "Top", "Worst", "Choice", "Original", "Favorite"]
    new_list_of_all_awards = []  

    # now only keep awards if they contain at least one word from keywords 
    for awards in all_awards:
        new_list_of_awards = []
        for award in awards:   

            for keyword in keywords:
                if keyword in award:
                    new_list_of_awards.append(award)
                    continue
        new_list_of_all_awards.append(new_list_of_awards)
    
    return new_list_of_all_awards    

In [6]:
standalones_df["Movie Awards"] = clean_awards(standalones_df, "Movie Awards")

In [7]:
standalones_df["budget"] = standalones_df["budget"].replace(0, np.nan)
standalones_df["budget"] = standalones_df["revenue"].replace(0, np.nan)

standalones_df["z_vote_average"] = stats.zscore(standalones_df["vote_average"])
standalones_df["z_vote_count"] = stats.zscore(standalones_df["vote_count"])
standalones_df["z_popularity"] = stats.zscore(standalones_df["popularity"])
standalones_df["z_budget"] = stats.zscore(standalones_df["budget"], nan_policy="omit")
standalones_df["z_revenue"] = stats.zscore(standalones_df["revenue"], nan_policy="omit")

standalones_df.columns

Index(['Unnamed: 0', 'title', 'imdb_id', 'id', 'release_date', 'runtime',
       'vote_average', 'vote_count', 'popularity', 'budget', 'revenue',
       'genres', 'original_language', 'production_companies',
       'production_countries', 'Movie Budget', 'Movie Box Office US/CA',
       'Movie Box Office Worldwide', 'Movie Length', 'Movie Rating',
       'Movie Director Names', 'Movie Director Links', 'Movie Cast Names',
       'Movie Cast Links', 'Movie Awards', 'z_vote_average', 'z_vote_count',
       'z_popularity', 'z_budget', 'z_revenue'],
      dtype='object')

In [8]:
def get_year(date_string):
    if type(date_string) == str:
        return date_string[0:4]
    return None

# replacing NaNs and empty strings with "0000-00-00" so it can be converted into a year
standalones_df["release_date"] = (standalones_df["release_date"].fillna("0000-00-00")
                           .replace(r'^\s*$', "0000-00-00", regex=True))

year_col = standalones_df["release_date"].apply(lambda x: get_year(x))
standalones_df.insert(4, "release_year", year_col)

# replacing missing values with NaN
standalones_df["release_year"] = standalones_df["release_year"].replace("0000", np.nan, regex=True)

In [14]:
def clean_list_of_dicts(list_of_dicts, dict_key):
    new_list = []
    for dict in list_of_dicts:
        new_list.append(dict[dict_key])
    return new_list    

standalones_df["genres"] = standalones_df["genres"].apply(lambda x: clean_list_of_dicts(eval(x), "name"))
standalones_df["production_companies"] = standalones_df["production_companies"].apply(lambda x: clean_list_of_dicts(eval(x), "name"))
standalones_df["production_countries"] = standalones_df["production_countries"].apply(lambda x: clean_list_of_dicts(eval(x), "name"))

standalones_df = standalones_df.rename(columns={"id": "tmdb_id"})

standalones_df.columns 

Index(['Unnamed: 0', 'title', 'imdb_id', 'tmdb_id', 'release_year',
       'release_date', 'runtime', 'vote_average', 'vote_count', 'popularity',
       'budget', 'revenue', 'genres', 'original_language',
       'production_companies', 'production_countries', 'Movie Budget',
       'Movie Box Office US/CA', 'Movie Box Office Worldwide', 'Movie Length',
       'Movie Rating', 'Movie Director Names', 'Movie Director Links',
       'Movie Cast Names', 'Movie Cast Links', 'Movie Awards',
       'z_vote_average', 'z_vote_count', 'z_popularity', 'z_budget',
       'z_revenue'],
      dtype='object')

In [20]:
standalones_df.to_csv("standalones_data.csv")

In [15]:
latex_df_1 = standalones_df[['title', 'imdb_id', 'tmdb_id', 'release_date', 'vote_average','vote_count', 'popularity']].head(10).style.to_latex()
latex_df_2 = standalones_df[['budget', 'genres', 'production_countries']].head(10).style.to_latex()
latex_df_3 = standalones_df[['Movie Box Office US/CA','Movie Box Office Worldwide','Movie Director Names']].head(10).style.to_latex()
latex_df_4 = standalones_df[['Movie Cast Names', 'Movie Awards']].head(10).style.to_latex()


In [16]:
print(latex_df_1)

\begin{tabular}{lllrlrrr}
 & title & imdb_id & tmdb_id & release_date & vote_average & vote_count & popularity \\
0 & A Clockwork Orange & tt0066921 & 185 & 1971-12-19 & 8.218000 & 11594 & 29.221000 \\
1 & Head-On & tt0347048 & 363 & 2004-03-11 & 7.453000 & 470 & 8.132000 \\
2 & Pi & tt0138704 & 473 & 1998-07-10 & 7.122000 & 1942 & 10.689000 \\
3 & Fat Girl & tt0243255 & 570 & 2001-03-07 & 6.215000 & 177 & 11.034000 \\
4 & Belle de Jour & tt0061395 & 649 & 1967-05-24 & 7.420000 & 738 & 17.263000 \\
5 & This Island Earth & tt0047577 & 831 & 1955-06-01 & 6.011000 & 190 & 9.733000 \\
6 & Doctor Zhivago & tt0059113 & 907 & 1965-12-22 & 7.556000 & 981 & 16.505000 \\
7 & Stagecoach & tt0031971 & 995 & 1939-03-03 & 7.680000 & 942 & 15.018000 \\
8 & Touch of Evil & tt0052311 & 1480 & 1958-03-30 & 7.774000 & 1234 & 13.884000 \\
9 & Population 436 & tt0462482 & 1596 & 2006-07-04 & 5.776000 & 225 & 9.928000 \\
\end{tabular}



In [17]:
print(latex_df_2)

\begin{tabular}{lrll}
 & budget & genres & production_countries \\
0 & 26589000.000000 & ['Science Fiction', 'Drama'] & ['United Kingdom', 'United States of America'] \\
1 & 11030861.000000 & ['Drama', 'Romance'] & ['Germany', 'Turkey'] \\
2 & 3221152.000000 & ['Mystery', 'Drama', 'Thriller'] & ['United States of America'] \\
3 & nan & ['Drama'] & ['France', 'Italy'] \\
4 & nan & ['Drama', 'Romance'] & ['France', 'Italy'] \\
5 & 1700000.000000 & ['Science Fiction', 'Adventure', 'Horror'] & ['United States of America'] \\
6 & 111858363.000000 & ['Drama', 'Romance', 'War'] & ['Italy', 'United Kingdom', 'United States of America'] \\
7 & 1103757.000000 & ['Western', 'Adventure', 'Drama'] & ['United States of America'] \\
8 & 2247500.000000 & ['Crime', 'Thriller', 'Mystery'] & ['United States of America'] \\
9 & nan & ['Horror', 'Thriller'] & ['Canada'] \\
\end{tabular}



In [18]:
print(latex_df_3)

\begin{tabular}{llll}
 & Movie Box Office US/CA & Movie Box Office Worldwide & Movie Director Names \\
0 & $26,617,553 & $26,960,374 & Stanley Kubrick \\
1 & $435,395 & $11,065,801 & Fatih Akin \\
2 & $3,221,152 & $3,221,152 & Darren Aronofsky \\
3 & $725,854 & $765,705 & Catherine Breillat \\
4 & $4,063,348 & $4,162,021 & Luis Buñuel \\
5 & nan & nan & Joseph M. Newman, Jack Arnold \\
6 & $111,721,910 & $111,922,142 & David Lean \\
7 & nan & nan & John Ford \\
8 & $2,247,465 & $2,285,063 & Orson Welles \\
9 & nan & nan & Michelle MacLaren \\
\end{tabular}



In [19]:
print(latex_df_4)

\begin{tabular}{lll}
 & Movie Cast Names & Movie Awards \\
0 & Malcolm McDowell, Patrick Magee, Michael Bates & [] \\
1 & Birol Ünel, Sibel Kekilli, Güven Kiraç & [] \\
2 & Sean Gullette, Mark Margolis, Ben Shenkman & [] \\
3 & Anaïs Reboux, Roxane Mesquida, Libero De Rienzo & [] \\
4 & Catherine Deneuve, Jean Sorel, Michel Piccoli & [] \\
5 & Jeff Morrow, Faith Domergue, Rex Reason & [] \\
6 & Omar Sharif, Julie Christie, Geraldine Chaplin & ['Best Writing', 'Best Cinematography', 'Best Art Direction-Set Decoration', 'Best Costume Design', 'Best Music', 'Score - Substantially Original', 'Best Picture', 'Best Actor in a Supporting Role', 'Best Director', 'Best Sound', 'Best Film Editing'] \\
7 & John Wayne, Claire Trevor, Andy Devine & [] \\
8 & Charlton Heston, Orson Welles, Janet Leigh & [] \\
9 & David Ames, Leigh Enns, Susan Kelso & [] \\
\end{tabular}

